In [ ]:
!pip -q install xgboost lightgbm catboost

import pandas as pd
import numpy as np

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

pd.set_option("display.width", 160)
pd.set_option("display.max_columns", 200)

# ✅ 최종 사용할 Feature (네 최고 기록 노트북 기준 20개)
final_feats = [
    'humidity', 'wind_gust_spd', 'hour', 'doy', 'wind_spd_b',
    'ceiling', 'uv_idx', 'appr_temp', 'uv_cloud_adj', 'dow',
    'hour_sin', 'doy_sin', 'is_rain', 'rain', 'hour_cos',
    'doy_cos', 'snow', 'coord1', 'coord2', 'haze'
]
target_col = "nins"

# 보간 파라미터(5분 간격 기준 12칸=1시간)
MAX_GAP = 12
DAY_HOURS = (6, 18)  # 주간 프록시
print("✅ Final feature count:", len(final_feats))


In [ ]:
def add_features(df: pd.DataFrame) -> pd.DataFrame:
    EPS = 1e-6
    out = df.copy()

    # time
    if not np.issubdtype(out["time"].dtype, np.datetime64):
        out["time"] = pd.to_datetime(out["time"])

    # 시간 파생
    out["hour"] = out["time"].dt.hour.astype("int16")
    out["dow"]  = out["time"].dt.dayofweek.astype("int16")
    out["doy"]  = out["time"].dt.dayofyear.astype("int16")
    out["hour_sin"] = np.sin(2*np.pi*out["hour"]/24).astype("float32")
    out["hour_cos"] = np.cos(2*np.pi*out["hour"]/24).astype("float32")
    out["doy_sin"]  = np.sin(2*np.pi*out["doy"]/365).astype("float32")
    out["doy_cos"]  = np.cos(2*np.pi*out["doy"]/365).astype("float32")

    # 강수 여부 (규칙 기반)
    if "precip_1h" in out.columns:
        out["is_rain"] = (pd.to_numeric(out["precip_1h"], errors="coerce").fillna(0) > 0).astype("int8")
    elif "rain" in out.columns:
        out["is_rain"] = (pd.to_numeric(out["rain"], errors="coerce").fillna(0) > 0).astype("int8")
    else:
        out["is_rain"] = 0

    # haze (가시거리 역수)
    if "vis" in out.columns:
        vis = pd.to_numeric(out["vis"], errors="coerce").astype("float32")
        out["haze"] = (1.0 / (vis + EPS)).astype("float32")
    else:
        out["haze"] = 0.0

    # pv_id 메모리 경감
    if "pv_id" in out.columns and not pd.api.types.is_categorical_dtype(out["pv_id"]):
        out["pv_id"] = out["pv_id"].astype("category")

    return out


In [ ]:
def interpolate_weather(df: pd.DataFrame, max_gap: int = MAX_GAP) -> pd.DataFrame:
    out = df.copy()

    # 주간 마스크
    day_mask = out["hour"].between(DAY_HOURS[0], DAY_HOURS[1])

    # ---- 이산/이벤트형: 보간 금지, 짧은 결손만 앞뒤로 메움 ----
    if "is_rain" in out.columns:
        s = out.groupby("pv_id", observed=True)["is_rain"].transform(
            lambda x: x.fillna(method="ffill").fillna(method="bfill")
        ).astype("float32")

        # 남은 NaN은 중앙값으로 단계적 채움
        if s.isna().any():
            s = s.fillna(out.groupby(["pv_id","hour"], observed=True)["is_rain"].transform("median"))
        if s.isna().any():
            s = s.fillna(out.groupby("pv_id", observed=True)["is_rain"].transform("median"))
        if s.isna().any():
            s = s.fillna(0)
        out["is_rain"] = s.astype("float32")

    # ---- 연속형: 선형 보간 + 중앙값 백업 ----
    cont_cols = [c for c in [
        "humidity", "wind_spd_b", "wind_gust_spd", "ceiling", "appr_temp", "uv_idx", "haze"
    ] if c in out.columns]

    for c in cont_cols:
        # uv_idx는 주간만 보간, 야간은 0 유지
        if c == "uv_idx":
            out[c] = pd.to_numeric(out[c], errors="coerce").astype("float32")
            out[c] = out[c].where(day_mask, 0.0)

        s_lin = out.groupby("pv_id", observed=True)[c].transform(
            lambda x: x.interpolate(method="linear", limit=max_gap, limit_direction="both")
        ).astype("float32")

        # 잔여 결측 단계적 대체: (pv,hour) → pv → 전체
        if s_lin.isna().any():
            s_lin = s_lin.fillna(out.groupby(["pv_id","hour"], observed=True)[c].transform("median"))
        if s_lin.isna().any():
            s_lin = s_lin.fillna(out.groupby("pv_id", observed=True)[c].transform("median"))
        if s_lin.isna().any():
            s_lin = s_lin.fillna(s_lin.median())

        out[c] = s_lin.astype("float32")

    # ---- rain/snow: 보간하지 않고 0으로 최소 대체 + 물리 클립 ----
    for c in ["rain", "snow"]:
        if c in out.columns:
            out[c] = pd.to_numeric(out[c], errors="coerce").fillna(0).clip(lower=0).astype("float32")

    # ---- 물리 범위 클립 ----
    clip_map = {
        "humidity": (0, 100),
        "uv_idx":   (0, None),
        "wind_spd_b": (0, None),
        "wind_gust_spd": (0, None),
        "ceiling": (0, None),
        "appr_temp": (None, None),  # 온도는 음수 가능
        "haze":    (0, None),
        "rain":    (0, None),
        "snow":    (0, None),
    }
    for c,(lo,hi) in clip_map.items():
        if c in out.columns:
            x = out[c].astype("float32")
            if lo is not None: x = np.maximum(x, lo)
            if hi is not None: x = np.minimum(x, hi)
            out[c] = x.astype("float32")

    # ---- uv_cloud_adj 재계산 ----
    EPS = 1e-6
    if "uv_idx" in out.columns:
        if "cloud_a" in out.columns:
            cloud_a_norm = pd.to_numeric(out["cloud_a"], errors="coerce").astype("float32")
            cloud_a_norm = cloud_a_norm / (float(cloud_a_norm.max()) + EPS)
            out["uv_cloud_adj"] = (out["uv_idx"] * (1 - cloud_a_norm)).astype("float32")
        else:
            out["uv_cloud_adj"] = out["uv_idx"].astype("float32")

    return out


In [ ]:
print("🚀 Loading train & test CSVs ...")
train_raw = pd.read_csv("train.csv", low_memory=True, memory_map=True)
test_raw  = pd.read_csv("test.csv",  low_memory=True, memory_map=True)
print(f"✅ Raw shapes — Train: {train_raw.shape} | Test: {test_raw.shape}")

train_raw = add_features(train_raw)
test_raw  = add_features(test_raw)

# 타깃 정제
train_raw[target_col] = pd.to_numeric(train_raw[target_col], errors="coerce").astype("float32")
before = len(train_raw)
train_raw = train_raw.dropna(subset=[target_col])
print(f"🔧 Dropped {before - len(train_raw)} rows with NaN target")


In [ ]:
train_df, valid_df = train_test_split(train_raw, test_size=0.2, random_state=42)

# 누수 방지: split 후 각각 보간
train_df = interpolate_weather(train_df, max_gap=MAX_GAP)
valid_df = interpolate_weather(valid_df, max_gap=MAX_GAP)

# 필요한 컬럼만 + NaN 최소 대체
train_df = train_df[[c for c in (final_feats + [target_col]) if c in train_df.columns]].copy()
valid_df = valid_df[[c for c in (final_feats + [target_col]) if c in valid_df.columns]].copy()
train_df[final_feats] = train_df[final_feats].astype("float32").fillna(0)
valid_df[final_feats] = valid_df[final_feats].astype("float32").fillna(0)

X_tr, y_tr = train_df[final_feats], train_df[target_col].values
X_va, y_va = valid_df[final_feats], valid_df[target_col].values
print("✅ Split OK —", f"train: {X_tr.shape}, valid: {X_va.shape}")


In [ ]:
# -----------------------
# 1) XGBoost (xgb.train)
# -----------------------
dtrain = xgb.DMatrix(X_tr, label=y_tr, feature_names=X_tr.columns.tolist())
dvalid = xgb.DMatrix(X_va, label=y_va, feature_names=X_va.columns.tolist())

xgb_params = {
    "objective": "reg:squarederror",
    "eval_metric": "mae",
    "learning_rate": 0.04,
    "max_depth": 7,
    "min_child_weight": 4,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "seed": 42,
    "tree_method": "hist",  # GPU면 "gpu_hist"
}

print("🚀 Training XGBoost...")
xgb_model = xgb.train(
    params=xgb_params,
    dtrain=dtrain,
    num_boost_round=4000,
    evals=[(dtrain,"train"),(dvalid,"valid")],
    early_stopping_rounds=200,
    verbose_eval=200
)
xgb_best_n = xgb_model.best_iteration + 1
pred_xgb_va = xgb_model.predict(dvalid, iteration_range=(0, xgb_model.best_iteration + 1))
mae_xgb = mean_absolute_error(y_va, pred_xgb_va)
print(f"✅ XGB best_iter={xgb_best_n}, valid MAE={mae_xgb:.4f}")

# -----------------------
# 2) LightGBM
# -----------------------
print("\n🚀 Training LightGBM...")
lgb_train = lgb.Dataset(X_tr, label=y_tr)
lgb_valid = lgb.Dataset(X_va, label=y_va, reference=lgb_train)

lgb_params = {
    "objective": "regression",
    "metric": "mae",
    "learning_rate": 0.04,
    "num_leaves": 64,
    "min_data_in_leaf": 50,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 1,
    "seed": 42,
}

lgb_model = lgb.train(
    params=lgb_params,
    train_set=lgb_train,
    num_boost_round=8000,
    valid_sets=[lgb_train, lgb_valid],
    valid_names=["train","valid"],
    callbacks=[lgb.early_stopping(200), lgb.log_evaluation(200)]
)
lgb_best_n = lgb_model.best_iteration
pred_lgb_va = lgb_model.predict(X_va, num_iteration=lgb_best_n)
mae_lgb = mean_absolute_error(y_va, pred_lgb_va)
print(f"✅ LGB best_iter={lgb_best_n}, valid MAE={mae_lgb:.4f}")

# -----------------------
# 3) CatBoost
# -----------------------
print("\n🚀 Training CatBoost...")
cat_model = CatBoostRegressor(
    loss_function="MAE",
    iterations=10000,
    learning_rate=0.04,
    depth=8,
    random_seed=42,
    verbose=200
)
cat_model.fit(X_tr, y_tr, eval_set=(X_va, y_va), use_best_model=True)
cat_best_n = cat_model.get_best_iteration()
pred_cat_va = cat_model.predict(X_va)
mae_cat = mean_absolute_error(y_va, pred_cat_va)
print(f"✅ CAT best_iter={cat_best_n}, valid MAE={mae_cat:.4f}")


In [ ]:
pred_mean3 = (pred_xgb_va + pred_lgb_va + pred_cat_va) / 3.0
mae_mean3 = mean_absolute_error(y_va, pred_mean3)
print(f"✅ Mean(3 models) valid MAE = {mae_mean3:.4f}")

# Weighted search: wx + wl + wc = 1, each >=0
# step 줄이면 더 정밀하지만 오래 걸림
step = 0.05

best = {"mae": 1e18, "wx": None, "wl": None, "wc": None}
grid = np.arange(0, 1 + 1e-9, step)

for wx in grid:
    for wl in grid:
        wc = 1.0 - wx - wl
        if wc < -1e-12:
            continue
        if wc < 0:
            wc = 0.0

        pred = wx * pred_xgb_va + wl * pred_lgb_va + wc * pred_cat_va
        mae = mean_absolute_error(y_va, pred)
        if mae < best["mae"]:
            best = {"mae": mae, "wx": float(wx), "wl": float(wl), "wc": float(wc)}

print(f"🏁 Best weighted(3 models): wx={best['wx']:.2f}, wl={best['wl']:.2f}, wc={best['wc']:.2f} | MAE={best['mae']:.4f}")

# 최종적으로 valid에서 가장 좋은 조합 선택
# (mean vs weighted 중 best)
use_weighted = best["mae"] <= mae_mean3
print("✅ Choose:", "WEIGHTED" if use_weighted else "MEAN")
